In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = pd.read_excel('C:/Users/Aravind/PROJECTS/Milky-way-of-Deep-Learning/Milky-way-of-Deep-Learning/data/external/full_dataset.xlsx')

In [4]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
data[['Diet']] = encoder.fit_transform(data[['Diet']])

In [5]:
X = data.drop(['Diet'] , axis=1)
y = data['Diet']

In [6]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

In [7]:
tf.random.set_seed(42)
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Flatten())
for layer in range(20):
    model1.add(tf.keras.layers.Dense(50, activation="relu",
                                    kernel_initializer="he_normal"))
model1.add(tf.keras.layers.Dense(3, activation="softmax"))

In [8]:
model1.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              metrics=["accuracy"])

In [11]:
history1 = model1.fit(X_train ,  y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0994 - accuracy: 0.3298 - val_loss: 1.0988 - val_accuracy: 0.3359
Epoch 2/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0992 - accuracy: 0.3170 - val_loss: 1.0982 - val_accuracy: 0.3389
Epoch 3/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3237 - val_loss: 1.0981 - val_accuracy: 0.3405
Epoch 4/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0990 - accuracy: 0.3415 - val_loss: 1.0990 - val_accuracy: 0.3130
Epoch 5/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3288 - val_loss: 1.0988 - val_accuracy: 0.3282
Epoch 6/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0991 - accuracy: 0.3379 - val_loss: 1.0991 - val_accuracy: 0.3115
Epoch 7/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3425 - val_loss: 1.0992 - val_accuracy: 0.3115
Epoch 8/20
62/62 [==

In [12]:
score = model1.evaluate(X_test, y_test)

21/21 [==============================] - 0s 953us/step - loss: 1.0988 - accuracy: 0.3405
